In [58]:
import numpy as np
import pandas as pd
import yfinance as yf


amzn = yf.download(tickers='AMZN',interval="1d",start='2023-01-01',end='2023-12-31')
print(amzn)


[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-01-03   85.459999   86.959999   84.209999   85.820000   85.820000   
2023-01-04   86.550003   86.980003   83.360001   85.139999   85.139999   
2023-01-05   85.330002   85.419998   83.070000   83.120003   83.120003   
2023-01-06   83.029999   86.400002   81.430000   86.080002   86.080002   
2023-01-09   87.459999   89.480003   87.080002   87.360001   87.360001   
...                ...         ...         ...         ...         ...   
2023-12-22  153.770004  154.350006  152.710007  153.419998  153.419998   
2023-12-26  153.559998  153.979996  153.029999  153.410004  153.410004   
2023-12-27  153.559998  154.779999  153.119995  153.339996  153.339996   
2023-12-28  153.720001  154.080002  152.949997  153.380005  153.380005   
2023-12-29  153.100006  153.889999  151.029999  151.940002  151.940002   

              Volume  
Date          

Trading Bot

In [59]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime 
from alpaca_trade_api import REST 
from timedelta import Timedelta 

from models.llmtime import get_llmtime_predictions_data
from finbert_utils import estimate_sentiment

import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage


os.environ['MISTRAL_KEY']='iyYT7uLWQZC4jDuVsKSVQ1tMcrZjc6s8'
mistral_client = MistralClient(os.environ['MISTRAL_KEY'])

API_KEY ="PKG9H92SIZ307T5C5YP2"
API_SECRET ="L17qx7T7MvslgQLM76Dzz1eXdui2YE27B7elENpo"
BASE_URL = "https://paper-api.alpaca.markets"

ALPACA_CREDS = {
    "API_KEY":API_KEY, 
    "API_SECRET": API_SECRET, 
    "PAPER": True
}

class MLTrader(Strategy): 
    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5,model=None): 
        self.symbol = symbol
        self.sleeptime = "24H" 
        self.last_trade = None 
        self.cash_at_risk = cash_at_risk
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)
        self.init_model_llm()

    def position_sizing(self): 
        cash = self.get_cash() 
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price,0)
        return cash, last_price, quantity
    
    def init_model_llm(self):
        self.model = MistralClient(os.environ['MISTRAL_KEY'])
    def get_sentiments_llm(self,news):
        mistral_sys_message = '''You a trading expert, you analyze news and provide sentiment analysis. the provided sentiment will be one word. The sentiment 
        can be one of three categories: positive, negative, or neutral.do not provide any explanation, just return one word.''' 
        extra_input = ''' Please analyse and summarize the following news articles about SPY index and provide a summurized trading sentiment of SPY index that summurize 
        all of the analysis  based on the information below, just return the one of the three sentiments : positive, negative or neutral,
        do not return any other sentence or word, just return only one word, do not provide explanation\n '''
        for i in range(len(news)):
            extra_input = extra_input + (str(i+1)+news[i]+"\n") 
        response = self.model.chat(
            model='mistral-tiny',
            messages=[ChatMessage(role="system", content = mistral_sys_message),ChatMessage(role="user", content= (extra_input))]
        )
        return [choice.message.content for choice in response.choices]
    def get_dates(self): 
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')

    def get_sentiment_llm(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start=three_days_prior, 
                                 end=today) 
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        sentiments_llm = self.get_sentiments_llm(news)
        return 1,  

    def get_sentiment(self): 
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start=three_days_prior, 
                                 end=today) 
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        sentiments_llm = self.get_sentiments_llm(news)
        print('\nSentiment LLM :' + str(sentiments_llm))
        probability, sentiment = estimate_sentiment(news)
        print('Sentiment Other : '+sentiment + ' ' + str(probability))
        return probability, sentiment 

    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing() 
        probability, sentiment = self.get_sentiment()

        if cash > last_price: 
            if sentiment == "positive" and probability > .999: 
                if self.last_trade == "sell": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol, 
                    quantity, 
                    "buy", 
                    type="bracket", 
                    take_profit_price=last_price*1.20, 
                    stop_loss_price=last_price*.95
                )
                self.submit_order(order) 
                self.last_trade = "buy"
            elif sentiment == "negative" and probability > .999: 
                if self.last_trade == "buy": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol, 
                    quantity, 
                    "sell", 
                    type="bracket", 
                    take_profit_price=last_price*.8, 
                    stop_loss_price=last_price*1.05
                )
                self.submit_order(order) 
                self.last_trade = "sell"
# trader = Trader()
# trader.add_strategy(strategy)
# trader.run_all()

In [ ]:
start_date = datetime(2023,12,1)
end_date = datetime(2023,12,5) 
broker = Alpaca(ALPACA_CREDS) 
strategy = MLTrader(name='mlstrat', broker=broker, 
                    parameters={"symbol":"SPY", 
                                "cash_at_risk":.5},model=mistral_client)
strategy.backtest(
    YahooDataBacktesting, 
    start_date, 
    end_date, 
    parameters={"symbol":"SPY", "cash_at_risk":.5}
)

Starting backtest for MLTrader...
Progress ||   9.90%  [Elapsed: 0:00:01 ETA: 0:00:11] 
Sentiment LLM :['Neutral. The articles present a mix of positive and negative news, such as the underperformance of tech stocks and the potential for a soft landing, as well as the investigation into the South Bay facility and the chilling outlook for the S&P 500 in 2024. The overall sentiment is neutral as there is no clear positive or negative trend.']
Sentiment Other : neutral tensor(0.9930, grad_fn=<SelectBackward0>)
Progress ||  84.91%  [Elapsed: 0:00:06 ETA: 0:00:01] Portfolio Val: 100,000.00
Sentiment LLM :['Neutral.\n\n(Please note that this sentiment analysis is based on the provided news articles and does not take into account other market factors or current market conditions.)']
Sentiment Other : negative tensor(0.7306, grad_fn=<SelectBackward0>)
Progress || 100.00%  [Elapsed: 0:00:08 ETA: 0:00:00] Portfolio Val: 100,000.00
Creating trades plot...

Creating indicators plot...

Creating te

{'cagr': 0.0,
 'volatility': 0.0,
 'sharpe': 0,
 'max_drawdown': {'drawdown': 0.0,
  'date': Timestamp('2023-12-02 09:30:00-0500', tz='America/New_York')},
 'romad': 0,
 'total_return': 0.0}

2024-02-04 10:57:00,386: alpaca.trading.stream: ERROR: error during websocket communication: [Errno 11001] getaddrinfo failed
Traceback (most recent call last):
  File "c:\Users\medjd\anaconda3\envs\llmfin\Lib\site-packages\alpaca\trading\stream.py", line 170, in _run_forever
    await self._start_ws()
  File "c:\Users\medjd\anaconda3\envs\llmfin\Lib\site-packages\alpaca\trading\stream.py", line 130, in _start_ws
    await self._connect()
  File "c:\Users\medjd\anaconda3\envs\llmfin\Lib\site-packages\alpaca\trading\stream.py", line 61, in _connect
    self._ws = await websockets.connect(self._endpoint, **self._websocket_params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\medjd\anaconda3\envs\llmfin\Lib\site-packages\websockets\legacy\client.py", line 659, in __await_impl_timeout__
    return await asyncio.wait_for(self.__await_impl__(), self.open_timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 